In [2]:
import pandas as pd
import pickle
from tqdm import tqdm
tqdm.pandas()

In [3]:
DIR_new = 'C:\\Users\\jesse\\Desktop\\Honors Project\\'

with open(DIR_new+ 'ya_reviews.pkl', 'rb') as f:
    reviews = pickle.load(f)

In [4]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2388654 entries, 0 to 2388653
Data columns (total 11 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   book_id                 int32 
 1   review_id               object
 2   rating                  int8  
 3   review_text             object
 4   n_votes                 int16 
 5   n_comments              int16 
 6   all_caps                int32 
 7   review_text_lemmatized  object
 8   tokenized               object
 9   bigrams                 object
 10  trigrams                object
dtypes: int16(2), int32(2), int8(1), object(6)
memory usage: 139.0+ MB


In [5]:
from langdetect import detect
from langdetect import DetectorFactory

DetectorFactory.seed = 0

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

mask = reviews['review_text'].apply(is_english)

# Use the mask to filter out non-English reviews
reviews = reviews[mask]

In [6]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2140850 entries, 0 to 2388653
Data columns (total 11 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   book_id                 int32 
 1   review_id               object
 2   rating                  int8  
 3   review_text             object
 4   n_votes                 int16 
 5   n_comments              int16 
 6   all_caps                int32 
 7   review_text_lemmatized  object
 8   tokenized               object
 9   bigrams                 object
 10  trigrams                object
dtypes: int16(2), int32(2), int8(1), object(6)
memory usage: 140.9+ MB


In [7]:
emotion_lex = pd.read_csv(DIR_new+ 'emotion_lex.csv')

In [8]:
emotion_lex

,lex_word,association,emotion
0,delight,syn,joyful
1,urge,syn,inspired
2,carnalize,syn,sensual
3,sensualize,syn,sensual
4,sympathize_with,syn,compassionate
...,...,...,...
2125,bucked_up,syn,inspired
2126,motivated,syn,inspired
2127,encouraged,syn,inspired
2128,focussed,syn,inspired


In [9]:
# def cleantokens(tokens):
#     final_words = []
#     for word in tokens:
#         final_words.append(word)
#     text = ''
#     for word in final_words:
#         text = text + word + ' '
#     return text

# def cleangrams(gram_list):
#     [' '.join(gram) for gram in gram_list]

# tester['token_review_text'] = tester['tokenized'].progress_apply(lambda x: cleantokens(x))

In [10]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize

# Create a dictionary where the keys are lex_words and the values are lists of associated emotions
emotion_dict = emotion_lex.groupby('lex_word')['emotion'].apply(list).to_dict()
emotion_dict

{'Bob_Hope': ['optimistic'],
 'Dame_Muriel_Spark': ['inspired'],
 'Hope': ['optimistic'],
 'John_Barleycorn': ['excited'],
 'Latin': ['sensual'],
 'Latinian_language': ['sensual'],
 'Leslie_Townes_Hope': ['optimistic'],
 'Mary_Jane': ['compassionate'],
 'Muriel_Sarah_Spark': ['inspired'],
 'Muriel_Spark': ['inspired'],
 'Romance': ['sensual'],
 'Romance_language': ['sensual'],
 'SOB': ['sad'],
 'Spark': ['inspired'],
 'Weird': ['uncomfortable'],
 'Wyrd': ['uncomfortable'],
 'abase': ['dissapointed'],
 'abash': ['uncomfortable'],
 'abashed': ['dissapointed', 'uncomfortable'],
 'abide': ['sad', 'dissapointed'],
 'abject': ['dissapointed'],
 'abode': ['nostalgic'],
 'abuse': ['offended'],
 'accent': ['anxious'],
 'accented': ['anxious'],
 'accentuate': ['anxious'],
 'accept': ['dissapointed'],
 'ache': ['offended', 'sad', 'nostalgic'],
 'aching': ['sad'],
 'achy': ['sad'],
 'acquit': ['dissapointed'],
 'act': ['inspired'],
 'activate': ['inspired'],
 'activated': ['excited'],
 'actuate': 

In [11]:

emotion_counts = Counter()

def get_emotions(text):
    emotion_counts = Counter()
    tokens = word_tokenize(text)
    for token in tokens:
        if token in emotion_dict:
            for emotion in emotion_dict[token]:
                emotion_counts[emotion] += 1
    return emotion_counts

In [12]:
tester = reviews.head(10)

In [13]:
tester['emotions'] = tester['review_text'].progress_apply(get_emotions)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 124.94it/s]
C:\Users\jesse\AppData\Local\Temp\ipykernel_24816\775867396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tester['emotions'] = tester['review_text'].progress_apply(get_emotions)


In [14]:
tester

,book_id,review_id,rating,review_text,n_votes,n_comments,all_caps,review_text_lemmatized,tokenized,bigrams,trigrams,emotions
0,2767052,248c011811e945eca861b5c31a549291,5,I cracked and finally picked this up. Very enj...,24,25,0,i crack and finally pick this up very enjoyabl...,"[crack, finally, pick, enjoyable, quick, not, ...","[(crack, finally), (finally, pick), (pick, enj...","[(crack, finally, pick), (finally, pick, enjoy...","{'compassionate': 6, 'nostalgic': 3, 'sad': 3,..."
1,23302416,84c0936a0f9868f38e75d2f9a5cb761e,5,I read this book because my fifth grade son wa...,0,0,0,i read this book because my fifth grade son be...,"[fifth, grade, son, require, glad, !, experien...","[(fifth, grade), (grade, son), (son, require),...","[(fifth, grade, son), (grade, son, require), (...",{'joyful': 1}
2,18053080,785c8db878f4009da9741dea51f641da,4,Though the book started out slow and only star...,0,0,0,though the book start out slow and only start ...,"[start, slow, start, interesting, towards, 100...","[(start, slow), (slow, start), (start, interes...","[(start, slow, start), (slow, start, interesti...","{'dissapointed': 1, 'confused': 1, 'sad': 1}"
3,17383543,34dc3c45d07e82718b05e73167259aef,2,"*Update - 10/27/13* - After some sleep, I thin...",0,0,0,after some sleep i think about allegiant overa...,"[sleep, think, allegiant, overall, without, in...","[(sleep, think), (think, allegiant), (allegian...","[(sleep, think, allegiant), (think, allegiant,...","{'anxious': 1, 'confused': 3, 'joyful': 1, 'se..."
4,16651458,d8d6b590780256fef7ae4a9550fe3e0d,5,"This is a moving, heartbreaking, view into a l...",0,0,0,this be a move heartbreaking view into a life ...,"[move, heartbreaking, view, life, obese, 16, o...","[(move, heartbreaking), (heartbreaking, view),...","[(move, heartbreaking, view), (heartbreaking, ...","{'inspired': 3, 'sad': 1, 'nostalgic': 2, 'exc..."
5,10782699,972ce1267de0213e3032c685386890e6,5,I never thought I would enjoy a zombie books w...,0,0,0,i never think i would enjoy a zombie book with...,"[never, think, enjoy, zombie, mix, romance, am...","[(never, think), (think, enjoy), (enjoy, zombi...","[(never, think, enjoy), (think, enjoy, zombie)...","{'nostalgic': 2, 'joyful': 1, 'sensual': 1, 'c..."
6,7260188,21063e8930ec28bc54e76f932aa30ce1,5,What a great ending to the trilogy!! The secon...,0,0,0,what a great end to the trilogy ! ! the second...,"[great, end, trilogy, !, !, second, good, awes...","[(great, end), (end, trilogy), (trilogy, !), (...","[(great, end, trilogy), (end, trilogy, !), (tr...","{'sad': 1, 'compassionate': 1, 'nostalgic': 1}"
7,6148028,a19c6c44a40a88c8ff825f40118a9c7c,5,LOVED IT!!,0,0,1,love it ! !,"[love, !, !]","[(love, !), (!, !)]","[(love, !, !)]",{}
8,2767052,c52e231744768e9d7f939d1cbeb87666,5,"Exciting, fun, entertaining! :)",0,0,0,excite fun entertaining !,"[excite, fun, entertaining, !]","[(excite, fun), (fun, entertaining), (entertai...","[(excite, fun, entertaining), (fun, entertaini...",{'joyful': 1}
9,13206828,1d5b64fe0408bb5e1824d14631ccb5ee,5,LET'S JUST SAY I AM SO THANKFUL I HAVE THE NEX...,0,0,1,let u just say i be so thankful i have the nex...,"[let, u, thankful, beside]","[(let, u), (u, thankful), (thankful, beside)]","[(let, u, thankful), (u, thankful, beside)]",{}


In [15]:
reviews['lex_emotions']=reviews['review_text'].progress_apply(get_emotions)

100%|██████████| 2140850/2140850 [2:18:29<00:00, 257.65it/s]    


In [16]:
reviews

,book_id,review_id,rating,review_text,n_votes,n_comments,all_caps,review_text_lemmatized,tokenized,bigrams,trigrams,lex_emotions
0,2767052,248c011811e945eca861b5c31a549291,5,I cracked and finally picked this up. Very enj...,24,25,0,i crack and finally pick this up very enjoyabl...,"[crack, finally, pick, enjoyable, quick, not, ...","[(crack, finally), (finally, pick), (pick, enj...","[(crack, finally, pick), (finally, pick, enjoy...","{'compassionate': 6, 'nostalgic': 3, 'sad': 3,..."
1,23302416,84c0936a0f9868f38e75d2f9a5cb761e,5,I read this book because my fifth grade son wa...,0,0,0,i read this book because my fifth grade son be...,"[fifth, grade, son, require, glad, !, experien...","[(fifth, grade), (grade, son), (son, require),...","[(fifth, grade, son), (grade, son, require), (...",{'joyful': 1}
2,18053080,785c8db878f4009da9741dea51f641da,4,Though the book started out slow and only star...,0,0,0,though the book start out slow and only start ...,"[start, slow, start, interesting, towards, 100...","[(start, slow), (slow, start), (start, interes...","[(start, slow, start), (slow, start, interesti...","{'dissapointed': 1, 'confused': 1, 'sad': 1}"
3,17383543,34dc3c45d07e82718b05e73167259aef,2,"*Update - 10/27/13* - After some sleep, I thin...",0,0,0,after some sleep i think about allegiant overa...,"[sleep, think, allegiant, overall, without, in...","[(sleep, think), (think, allegiant), (allegian...","[(sleep, think, allegiant), (think, allegiant,...","{'anxious': 1, 'confused': 3, 'joyful': 1, 'se..."
4,16651458,d8d6b590780256fef7ae4a9550fe3e0d,5,"This is a moving, heartbreaking, view into a l...",0,0,0,this be a move heartbreaking view into a life ...,"[move, heartbreaking, view, life, obese, 16, o...","[(move, heartbreaking), (heartbreaking, view),...","[(move, heartbreaking, view), (heartbreaking, ...","{'inspired': 3, 'sad': 1, 'nostalgic': 2, 'exc..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2388649,6137154,1f6fc20b8187e8f36e12daf947b13e66,5,A really amazing book! It goes great with Grac...,0,0,0,a really amazing book ! it go great with grace...,"[really, amazing, !, great, graceling, connect...","[(really, amazing), (amazing, !), (!, great), ...","[(really, amazing, !), (amazing, !, great), (!...",{}
2388650,10193062,fc86ebe129a7f14a09c0221c92f52f79,3,This book is better then the first. It has it'...,0,0,0,this book be well then the first it have it be...,"[slow, point, mostly, romance, section, not, n...","[(slow, point), (point, mostly), (mostly, roma...","[(slow, point, mostly), (point, mostly, romanc...","{'sad': 4, 'dissapointed': 3, 'sensual': 1, 'a..."
2388651,6186357,15d0f64aa991f270c490e1a4d5b4011b,2,I think this may have come from having expecta...,0,0,0,i think this may have come from have expectati...,"[think, may, expectation, set, heard, expect, ...","[(think, may), (may, expectation), (expectatio...","[(think, may, expectation), (may, expectation,...","{'sad': 1, 'dissapointed': 1, 'nostalgic': 2, ..."
2388652,14740456,b048505c5b1e9ab8695afa3edce1b5d9,4,I generally liked the book. It had some parts ...,0,0,0,i generally like the book it have some part th...,"[generally, like, felt, obvious, expect, many,...","[(generally, like), (like, felt), (felt, obvio...","[(generally, like, felt), (like, felt, obvious...","{'nostalgic': 2, 'compassionate': 1, 'sad': 1,..."


In [17]:
#export to pickle
with open(DIR_new+ 'reviews_Lex_analyzed.pkl', 'wb') as f:
    pickle.dump(reviews, f)

In [18]:
#export to csv
reviews.to_csv(DIR_new+ 'reviews_lex_analyzed.csv', index=False)